In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [3]:
df=pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
sub_df=pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")

In [4]:
sub_df

,id,NObeyesdad
0,20758,Normal_Weight
1,20759,Normal_Weight
2,20760,Normal_Weight
3,20761,Normal_Weight
4,20762,Normal_Weight
...,...,...
13835,34593,Normal_Weight
13836,34594,Normal_Weight
13837,34595,Normal_Weight
13838,34596,Normal_Weight


In [44]:
df

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


In [45]:
df.isnull().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [46]:
df.nunique()

id                                20758
Gender                                2
Age                                1703
Height                             1833
Weight                             1979
family_history_with_overweight        2
FAVC                                  2
FCVC                                934
NCP                                 689
CAEC                                  4
SMOKE                                 2
CH2O                               1506
SCC                                   2
FAF                                1360
TUE                                1297
CALC                                  3
MTRANS                                5
NObeyesdad                            7
dtype: int64

In [47]:
df.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [48]:
columns_to_transform=["Gender","family_history_with_overweight","FAVC","CAEC","SMOKE","SCC","CALC","MTRANS"]
columns_to_transform

['Gender',
 'family_history_with_overweight',
 'FAVC',
 'CAEC',
 'SMOKE',
 'SCC',
 'CALC',
 'MTRANS']

In [49]:
# categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

In [50]:
transform=ColumnTransformer(transformers=[
    ("text",OneHotEncoder(),columns_to_transform)],
     remainder='passthrough')

In [51]:
transform_data=transform.fit_transform(df)

In [52]:
transformed_columns = transform.named_transformers_['text'].get_feature_names_out(columns_to_transform)

# Create a DataFrame with the transformed columns
transformed_df = pd.DataFrame(transform_data, columns=list(transformed_columns) + [col for col in df.columns if col not in columns_to_transform])

# Print the transformed DataFrame
print(transformed_df)

      Gender_Female Gender_Male family_history_with_overweight_no  \
0               0.0         1.0                               0.0   
1               1.0         0.0                               0.0   
2               1.0         0.0                               0.0   
3               1.0         0.0                               0.0   
4               0.0         1.0                               0.0   
...             ...         ...                               ...   
20753           0.0         1.0                               0.0   
20754           0.0         1.0                               1.0   
20755           0.0         1.0                               0.0   
20756           0.0         1.0                               0.0   
20757           0.0         1.0                               0.0   

      family_history_with_overweight_yes FAVC_no FAVC_yes CAEC_Always  \
0                                    1.0     0.0      1.0         0.0   
1                        

In [53]:
transformed_df.columns

Index(['Gender_Female', 'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'SMOKE_no', 'SMOKE_yes', 'SCC_no', 'SCC_yes', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile', 'MTRANS_Bike',
       'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking',
       'id', 'Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'NObeyesdad'],
      dtype='object')

In [54]:
X2=transformed_df.drop(columns=["id","NObeyesdad"])
Y=transformed_df[["NObeyesdad"]]
label_encod=LabelEncoder()
Y2=label_encod.fit_transform(Y)
label_dataframe=pd.DataFrame(Y2)
label_dataframe["NObeyesdad"]=df[['NObeyesdad']]
label_dataframe

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0,NObeyesdad
0,6,Overweight_Level_II
1,1,Normal_Weight
2,0,Insufficient_Weight
3,4,Obesity_Type_III
4,6,Overweight_Level_II
...,...,...
20753,3,Obesity_Type_II
20754,0,Insufficient_Weight
20755,3,Obesity_Type_II
20756,6,Overweight_Level_II


In [55]:
df.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [57]:
X_train1,X_test1,Y_train1,Y_test1=train_test_split(X2,Y2,test_size=0.2,random_state=42)

In [62]:
base_estimator=[
    ("gradient_regre",GradientBoostingClassifier()),
    ("adaboost_reg",AdaBoostClassifier()),
    ("bagging",BaggingClassifier())
]
stack_model=StackingClassifier(estimators=base_estimator,final_estimator=RandomForestClassifier())

In [63]:
stack_model.fit(X_train1,Y_train1)

StackingClassifier(estimators=[('gradient_regre', GradientBoostingClassifier()),
                               ('adaboost_reg', AdaBoostClassifier()),
                               ('bagging', BaggingClassifier())],
                   final_estimator=RandomForestClassifier())

In [64]:
stack_model.score(X_test1,Y_test1)

0.9024566473988439

In [84]:
y_prob=stack_model.predict(X_test1)

# # confusion_matrix

In [88]:
cm = confusion_matrix(Y_test1, y_pred)
print(cm)

array([[485,  39,   0,   0,   0,   0,   0],
       [110, 516,   0,   0,   0,   0,   0],
       [541,   2,   0,   0,   0,   0,   0],
       [657,   0,   0,   0,   0,   0,   0],
       [804,   0,   0,   0,   0,   0,   0],
       [447,  37,   0,   0,   0,   0,   0],
       [503,  11,   0,   0,   0,   0,   0]])

# 
> # **Put train input data**

In [66]:

s1=df.iloc[[110]]
s1

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
110,110,Male,22.0,1.8,95.0,yes,yes,2.0,3.0,Sometimes,no,3.0,no,3.0,2.0,Sometimes,Public_Transportation,Obesity_Type_I


In [73]:
input_data=(s1)
transform_data2=transform.transform(input_data)
transformed_columns = transform.named_transformers_['text'].get_feature_names_out(columns_to_transform)

# Create a DataFrame with the transformed columns
transformed_df2 = pd.DataFrame(transform_data2, columns=list(transformed_columns) + [col for col in df.columns if col not in columns_to_transform])

# Print the transformed DataFrame

new_data=transformed_df2.drop(columns=["id","NObeyesdad"])
# print(new_data)
# array_input=np.array(new_data)
# array_reshape=array_input.reshape(1,-1)
y_pred2=stack_model.predict(new_data)
y_pred2


array([2])

# # put test input data

In [79]:
# d=df_test.head(1)
df_test['NObeyesdad']=0
s3=df_test.sample(1)
# //put no of row here
s3=df_test.iloc[[5]]
s3

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
5,20763,Male,19.799054,1.844751,59.605028,yes,yes,2.0,4.0,Sometimes,no,2.722063,no,2.0,1.283673,no,Automobile,0


In [82]:
input_data3=(s3)
transform_data3=transform.transform(input_data3)
transformed_columns3 = transform.named_transformers_['text'].get_feature_names_out(columns_to_transform)

# Create a DataFrame with the transformed columns
transformed_df3 = pd.DataFrame(transform_data3, columns=list(transformed_columns3) + [col for col in df.columns if col not in columns_to_transform])

# Print the transformed DataFrame
new_data3=transformed_df3.drop(columns=["id","NObeyesdad"])
y_pred3=stack_model.predict(new_data3)
y_pred3

array([0])

# # check output meaning

In [126]:
# label_dataframe/
print(label_dataframe[0].unique())
print(label_dataframe["NObeyesdad"].unique())

[6 1 0 4 3 5 2]
['Overweight_Level_II' 'Normal_Weight' 'Insufficient_Weight'
 'Obesity_Type_III' 'Obesity_Type_II' 'Overweight_Level_I'
 'Obesity_Type_I']


In [92]:
print(type(Y_test1), type(y_pred))
print(Y_test1[:5], y_pred2[:5])  # Print first 5 elements to inspect


<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[4 5 2 3 0] [2]
